### Подключаемся к Spark

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [9]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("Morozov_Nikita")
         .getOrCreate())

In [10]:
spark

### Посмотрим, какие файлы есть

In [11]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


### u.data содержит все оценки(Структура: user id | item id | rating | timestamp)

### Создадим rdd на основе файла u.data

In [12]:
rdd_u_data = spark.sparkContext.textFile('/labs/laba01/ml-100k/u.data')

### Расчитаем количество записей

In [13]:
rdd_u_data.count()

100000

### Посмотрим, что из себя представляют данные

In [14]:
rdd_u_data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

### Сплитанем rdd

In [15]:
rdd_u_data_split = rdd_u_data.map(lambda x: x.split("\t"))

In [16]:
rdd_u_data_split.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [21]:
rdd_test = rdd_u_data_split_two_columns = rdd_u_data_split.map(lambda x: (x[1]))

In [23]:
rdd_test.take(5)

['242', '302', '377', '51', '346']

In [40]:
rdd_test.filter(lambda x: x=='242').collect()

['242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242',
 '242']

### Выберем только нужные столбцы для будущих рассчетов

In [50]:
rdd_u_data_split_two_columns = rdd_u_data_split.map(lambda x: (x[1], x[2]))

In [51]:
rdd_u_data_split_two_columns.take(10)

[('242', '3'),
 ('302', '3'),
 ('377', '1'),
 ('51', '2'),
 ('346', '1'),
 ('474', '4'),
 ('265', '2'),
 ('465', '5'),
 ('451', '3'),
 ('86', '3')]

### HIST-ALL(Расчет количества уникальных оценок пользователей)

In [103]:
HIST_ALL = rdd_u_data_split_two_columns.map(lambda x: (x[1])).countByKey().items()

In [104]:
HIST_ALL = sorted(HIST_ALL)

In [105]:
lst_all = []
for _, quantity in HIST_ALL:
    lst_all.append(quantity)

In [106]:
print(HIST_ALL)
print(lst_all)

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]
[6110, 11370, 27145, 34174, 21201]


### HIST-FILM((Расчет количества уникальных оценок пользователей по фильму 302)

In [107]:
HIST_FILM = rdd_u_data_split_two_columns.filter(lambda x: x[0] == '302').map(lambda x: (x[1])).countByKey().items()

In [108]:
HIST_FILM = sorted(HIST_FILM)

In [109]:
lst_film = []
for _, quantity in HIST_FILM:
    lst_film.append(quantity)

In [110]:
print(HIST_FILM)
print(lst_film)

[('1', 2), ('2', 10), ('3', 46), ('4', 119), ('5', 120)]
[2, 10, 46, 119, 120]


### Запись в json формат

In [111]:
dict_for_json = {'hist_film':lst_film, 'hist_all':lst_all}

In [112]:
dict_for_json

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [114]:
import json
with open("lab01.json", "w") as outfile:
    json.dump(dict_for_json, outfile)

### Остановка Spark сессии

In [115]:
spark.stop()